### Import Packages

In [1]:
import sys
sys.path.append(r'R:\JoePriceResearch\record_linking\projects\deep_learning\ml-record-linking\splycer')
from splycer.record_set import RecordDataFrame
from splycer.pairs_set import PairsCOO, PairsMatrix
from splycer.feature_engineer import FeatureEngineer
from splycer.xgboost_match import XGBoostMatch
from splycer.record_set import RecordDB

from xgboost import XGBClassifier
import pickle as pkl
import pandas as pd
import pyodbc
import traceback

AttributeError: module 'bottleneck' has no attribute '__version__'

### Feature Engineer

In [2]:
print("creating feature engineer")
fe = FeatureEngineer()
cols = ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county",
        "immigration", "birth_year", ["res_lat", "res_lon"], ["bp_lat", "bp_lon"],
        [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)],
        "first", "last",
        "first", "last",
        "first", "last",
        "first", "last"
       ]


#similarity functions
col_comps = ["exact match"] * 9
col_comps.extend(["abs dist"] * 2)
col_comps.extend(["geo dist"] * 2)
col_comps.extend(["euclidean dist"] * 2)
col_comps.extend(["jw"] * 2)
col_comps.extend(["trigram"] * 2)
col_comps.extend(["bigram"] * 2)
col_comps.extend(['ngram'] * 2)
#extra arguments
col_args = list({} for i in range(5))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"},
                 {"comm_weight": "d", "comm_col": "bp_comm"}])
col_args.extend(list({} for i in range(7)))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"}] * 4)

assert len(cols) == len(col_comps) == len(col_args)
for i, j, k in zip(cols, col_comps, col_args):
    fe.add_comparison(i,j,k)

creating feature engineer


### Load Data

In [3]:
from tqdm import tqdm
n=100000
chunk=10000
conn = pyodbc.connect("DSN=rec_db")
print("Loading data...")

chunks = pd.read_sql("Select top 10000 * FROM Price.dbo.AA_training_data_1910_1920_large", conn,chunksize=chunk)
data=[]
for chunk in tqdm(chunks):
    data.append(chunk)
df=pd.concat(data)


Loading data...


1it [00:00, 23.32it/s]


### Structure Data

In [4]:
'''
#Build record sets
print("creating record set 1")
cols1910 = [s for s in df.columns if "1910" in s]
df1910 = df[cols1910].drop_duplicates()
df1910.columns = [s.replace("_1910", "") for s in df1910.columns]
record_set1 = RecordDataFrame(1910, df1910.set_index("index", drop=True))
del df1910

print("creating record set 2")
cols1920 = [s for s in df.columns if "1920" in s and s != "true_index_1920"]
df1920 = df[cols1920].drop_duplicates()
df1920.columns = [s.replace("_1920", "")for s in df1920.columns]
record_set2 = RecordDataFrame(1920, df1920.set_index("index", drop=True))
del df1920
'''

# Build record sets
record_set1=RecordDB(1910,'dbo.AA_training_data_1910_compiled','[index]','rec_db')
record_set2=RecordDB(1920,'dbo.AA_training_data_1920_compiled','[index]','rec_db')

# Build pairs sets
print("creating pairs set")
uids1 = df["index_1910"]
uids2 = df["index_1920"]
is_match = (df["index_1920"] == df["true_index_1920"])

# create holdout set for test validation
idx=is_match.iloc[:178].index
# create pairs set containing compares for the holdoutset only
sample_pair_set=PairsCOO(1910,1920,uids1.iloc[idx],uids2.iloc[idx],is_match.iloc[idx])

for frame in [uids1,uids2,is_match]:
    # remove holdout set data from training data
    frame.drop(idx,axis=0,inplace=True)
    
pairs_set = PairsCOO(1910, 1920, uids1, uids2, is_match)

creating pairs set


# # FIXME

In [5]:
import turbodbc
import numpy as np
import time
maxsize=1000000
nrows = sample_pair_set.ncompares
if maxsize < sample_pair_set.ncompares:
    nrows = maxsize
uids1 = []
uids2 = []
labels = []
i = 1
for uid1, uid2, label in sample_pair_set:
    uids1.append(uid1)
    uids2.append(uid2)
    labels.append(label)
    if i > maxsize:
        break
    i += 1

extra_joins = """
self.cursor.execute(f"select column_name from information_schema.columns where table_name = '{self.table_name}'")
cols = self.cursor.fetchall()
self.cols = np.ndarray(len(cols), dtype="U50")
for i in range(len(cols)):
    self.cols[i] = cols[i][0]
"""
table_name='dbo.AA_training_data_1910_compiled'
idx_name='[index]'
dsn='rec_db'
conn = turbodbc.connect(dsn='rec_db', turbodbc_options=turbodbc.make_options(prefer_unicode=True))
data = pd.read_sql(f"select * from {table_name} where {idx_name} in {tuple(uids1)}", conn)
'''
cursor=conn.cursor()
cursor.execute(f"select column_name from INFORMATION_SCHEMA.columns where table_name = '{table_name}'")
cols = self.cursor.fetchall()
'''

df1=record_set1.get_records(uids1)
df2=record_set2.get_records(uids2)
from pandas.util.testing import assert_frame_equal
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)
checkEqual(df1.columns.values,df2.columns.values)
#pairs1=uids1.values
#pairs2=uids2.values

'''
for label in ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county", "immigration", "birth_year"]:
    (df1[label] == df2[label]).mask(df1[label].isna() | df2[label].isna(), np.nan)
'''
record_set1.extra_joins

''

In [6]:
set1=set(df1['index'].tolist())
set2=set(df2['index'].tolist())
set1a=tuple(uids1)
set2a=set(uids2)
print(f'set1={len(set1)} set2={len(set2)}')
print(f'set1a={len(set1a)} set2a={len(set2a)}')

set1=177 set2=178
set1a=178 set2a=178


In [7]:
# df1.set_index('index').loc[uids1].reset_index()
# df2.set_index('index').loc[uids2].reset_index()
df1.head()

,index,marstat,birth_year,household,immigration,race,rel,female,bp,mbp,...,last_vec192,last_vec193,last_vec194,last_vec195,last_vec196,last_vec197,last_vec198,last_vec199,last_vec200,last_vec201
0,9940583,0.0,1906,2484165.0,NaN,1,3,1,24,24,...,-0.252548,0.182037,0.059878,0.537467,0.204997,-0.671293,-0.090781,-0.354124,0.127814,-0.386039
1,240916,0.0,1910,71695.0,NaN,1,3,1,3,3,...,-0.031697,0.034178,0.012270,0.088089,-0.271884,-0.149222,-0.500562,-0.688743,0.211320,-0.307028
2,24830219,1.0,1884,5704274.0,NaN,1,1,1,32,41,...,0.277734,-0.054636,0.089288,0.266973,0.161947,-0.363966,-0.109247,-0.009724,-0.282315,0.417428
3,77103589,0.0,1903,10317375.0,NaN,1,3,1,50,50,...,0.024284,0.062866,-0.376699,-0.241264,-0.101558,-0.252486,0.175191,-0.277608,-0.089618,-0.485440
4,37617749,1.0,1857,19761904.0,NaN,1,1,1,65,65,...,0.005522,-0.139591,-0.000388,-0.062933,-0.127447,-0.262847,0.217197,-0.001719,-0.127425,-0.008389


In [8]:
import pickle as pkl
with open(f"temp_model", 'wb') as file:
    pkl.dump(model,file)

NameError: name 'model' is not defined

### Create Model Objects

In [9]:
#training initial model:
model = XGBClassifier(n_estimators=2000, max_depth=3, learning_rate=.01, n_jobs=8,importance_type='gain')
print("creating xgb match object")
xgb_trainer = XGBoostMatch(record_set1, record_set2, sample_pair_set, fe, model)
print("training")
xgb_trainer.train()

creating xgb match object
training
creating training set...


ValueError: Can only compare identically-labeled Series objects

### Evalulate Model

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
def evalulate(pair_set):
    xgb_predictor=XGBoostMatch(record_set1,record_set2,pair_set,fe,model)
    X,y=xgb_predictor.create_training_set()
    y_pred = xgb_predictor.model.predict(X)
    confusion_mat = confusion_matrix(y, y_pred)
    test_precision = precision_score(y, y_pred)
    test_recall = recall_score(y, y_pred)
    return confusion_mat,test_precision,test_recall
print(evalulate(sample_pair_set))

In [ ]:
# FIXME fix save prediction results to txt file thing
# located in XGBoostMatch class
from datetime import date
today = date.today()
xgb_predictor=XGBoostMatch(record_set1,record_set2,sample_pair_set,fe,model)
xgb_predictor.run(f'AA_model_{today.strftime("%m-%d-%y")}_sample_test.txt')

In [ ]:
! conda list

In [ ]:
from datetime import date
today = date.today()
file_name=f'V:\projects\deep_learning\AA_model\AA_model_{today.strftime("%m-%d-%y")}'
pkl.dump(xgb_trainer.model, open(file_name, "wb"))

### Test Logit

In [ ]:
# FIXME: sparse data can't be used in logreg
try:
    import numpy as np
    col_names=np.array(["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county",
            "immigration", "birth_year", 'res_lat_lon', 'bp_lat_lon',
            'first_vec', 'last_vec',
            "first_jaro", "last_jaro",
            "first_trigram_comm", "last_trigram_comm",
            "first_bigram_comm", "last_bigram_comm",
            "first_ngram_comm", "last_ngram_comm"
            ])


    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split

    X,y=xgb_match.create_training_set()
    X=np.nan_to_num(X,0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=69)
    log=LogisticRegression()

    from sklearn.metrics import confusion_matrix, precision_score, recall_score

    log.fit(X_train,y_train)
    y_pred = log.predict(X_test)

    confusion_mat = confusion_matrix(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)

    print(f"number of training obs: {X.shape[0]},\n\
            precision: {test_precision}, recall: {test_recall}")
except:
    pass